In [2]:
%load_ext autoreload
%autoreload 2

%load_ext rpy2.ipython

D:\IT\Coding\GitHub Projects\My-University\III Курс\(python) Эконометрика\venv\Lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [3]:
from __future__ import annotations

import pandas as pd
import statsmodels.api as sm
from IPython.display import Markdown
from scipy import stats
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import reset_ramsey

# Задание 1

In [4]:
ALPHA = 0.05

In [ ]:
df = pd.read_csv('data/data.csv', sep=';', decimal=',')
df.head()

In [6]:
df.corr()

,y,x1,x2,x3,x4
y,1.000000,0.701097,0.690454,0.067950,-0.794822
x1,0.701097,1.000000,0.509960,0.255408,-0.460771
x2,0.690454,0.509960,1.000000,-0.385527,-0.560071
x3,0.067950,0.255408,-0.385527,1.000000,-0.166677
x4,-0.794822,-0.460771,-0.560071,-0.166677,1.000000


## Модель $ y = \alpha + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 + \beta_4 x_4 + \varepsilon $

In [7]:
y = df['y']
X = df[['x1', 'x2', 'x3', 'x4']]
X['const'] = 1

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.91, shuffle=False)
X_test

,x1,x2,x3,x4,const
20,5410.0,30.92,112.0,6.6,1
21,5023.0,31.53,110.8,6.5,1


In [9]:
main_model = sm.OLS(y_train, X_train)
main_results = main_model.fit()

In [10]:
print(main_results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.906
Model:                            OLS   Adj. R-squared:                  0.881
Method:                 Least Squares   F-statistic:                     36.15
Date:                Sat, 11 Dec 2021   Prob (F-statistic):           1.55e-07
Time:                        02:33:20   Log-Likelihood:                -144.58
No. Observations:                  20   AIC:                             299.2
Df Residuals:                      15   BIC:                             304.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.4473      0.130      3.430      0.0

## Расчет ошибки прогноза

In [11]:
y_pred_test_intervals = main_results.get_prediction(X_test).summary_frame(ALPHA)
y_pred_test_intervals

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
20,14250.571960,126.652990,13980.617503,14520.526417,13385.995225,15115.148696
21,14155.262839,135.924016,13865.547656,14444.978022,13284.313696,15026.211982


In [12]:
print('Для среднего')
all((y_pred_test_intervals['mean_ci_lower'] <= y_test) & (y_test <= y_pred_test_intervals['mean_ci_upper']))  # noqa

Для среднего


False

In [13]:
print('Для индивидуального')
all((y_pred_test_intervals['obs_ci_lower'] <= y_test) & (y_test <= y_pred_test_intervals['obs_ci_upper']))  # noqa

Для индивидуального


False

## Модель $ y = \alpha + \beta_1 x_1 + \beta_2 x_2 + \beta_4 x_4 + \varepsilon $

In [14]:
X124 = X[['x1', 'x2', 'x4', 'const']]

In [15]:
X124_train, X124_test, y_train, y_test = train_test_split(X124, y, train_size=0.91, shuffle=False)

In [16]:
model_124 = sm.OLS(y_train, X124_train)
results_124 = model_124.fit()

In [17]:
print(results_124.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.879
Method:                 Least Squares   F-statistic:                     47.05
Date:                Sat, 11 Dec 2021   Prob (F-statistic):           3.67e-08
Time:                        02:33:27   Log-Likelihood:                -145.38
No. Observations:                  20   AIC:                             298.8
Df Residuals:                      16   BIC:                             302.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.3716      0.112      3.310      0.0

In [18]:
y_pred_test_intervals = results_124.get_prediction(X124_test).summary_frame(ALPHA)
y_pred_test_intervals

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
20,14275.686143,125.609252,14009.406423,14541.965863,13410.459547,15140.912739
21,14228.003972,120.250711,13973.083853,14482.924092,13366.205596,15089.802348


In [19]:
print('Для среднего')
all((y_pred_test_intervals['mean_ci_lower'] <= y_test) & (y_test <= y_pred_test_intervals['mean_ci_upper']))  # noqa

Для среднего


False

In [20]:
print('Для индивидуального')
all((y_pred_test_intervals['obs_ci_lower'] <= y_test) & (y_test <= y_pred_test_intervals['obs_ci_upper']))  # noqa

Для индивидуального


False

## Модель $ y = \alpha + \beta_1 x_1 + \beta_4 x_4 + \varepsilon $

In [21]:
X14 = X[['x1', 'x4', 'const']]

In [22]:
X14_train, X14_test, y_train, y_test = train_test_split(X14, y, train_size=0.91, shuffle=False)

In [23]:
model_14 = sm.OLS(y_train, X14_train)
results_14 = model_14.fit()

In [24]:
print(results_14.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.879
Model:                            OLS   Adj. R-squared:                  0.864
Method:                 Least Squares   F-statistic:                     61.59
Date:                Sat, 11 Dec 2021   Prob (F-statistic):           1.63e-08
Time:                        02:33:30   Log-Likelihood:                -147.13
No. Observations:                  20   AIC:                             300.3
Df Residuals:                      17   BIC:                             303.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.4336      0.113      3.844      0.0

In [25]:
y_pred_test_intervals = results_14.get_prediction(X14_test).summary_frame(ALPHA)
y_pred_test_intervals

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
20,14242.075714,131.432766,13964.776816,14519.374611,13331.321865,15152.829562
21,14107.551642,104.374703,13887.340267,14327.763017,13212.525975,15002.577309


In [26]:
print('Для среднего')
all((y_pred_test_intervals['mean_ci_lower'] <= y_test) & (y_test <= y_pred_test_intervals['mean_ci_upper']))  # noqa

Для среднего


False

In [27]:
print('Для индивидуального')
all((y_pred_test_intervals['obs_ci_lower'] <= y_test) & (y_test <= y_pred_test_intervals['obs_ci_upper']))  # noqa

Для индивидуального


False

## Тест Чоу для изначальной модели

In [113]:
from dataclasses import dataclass


@dataclass
class ChowResults:
    value: float
    f_critical: float
    f_pvalue: float


# H0: выборка однородна (может быть описана общей моделью)
# H1: выборка не однородна

def chow_test(x: pd.DataFrame | pd.Series, y: pd.Series, splitter, alpha: float = 0.05) -> ChowResults:
    x_1, x_2, y_1, y_2 = splitter(x, y)

    def get_results(x, y) -> sm.regression.linear_model.RegressionResults:
        return sm.OLS(y, x).fit()

    main_results = get_results(x, y)
    results_1 = get_results(x_1, y_1)
    results_2 = get_results(x_2, y_2)

    rss = main_results.ssr
    rss_1 = results_1.ssr
    rss_2 = results_2.ssr

    k = main_results.df_model
    n_1 = results_1.nobs
    n_2 = results_2.nobs

    chow = (rss - rss_1 - rss_2) * (n_1 + n_2 - 2 * (k + 1)) / (rss_1 + rss_2) / (k + 1)
    f_critical = stats.f.ppf(1 - alpha, k + 1, n_1 + n_2 - 2 * (k + 1))
    f_pvalue = 1 - stats.f.cdf(chow, dfn=k + 1, dfd=(n_1 + n_2 - 2 * k))

    return ChowResults(chow, f_critical, f_pvalue)

In [114]:
# H0 принимается - выборки однородны
chow_stats = chow_test(X_train, y_train, lambda x, y: train_test_split(x, y, train_size=0.5, shuffle=False))
chow_stats

ChowResults(value=0.2859018806223697, f_critical=3.259166726901249, f_pvalue=0.8822488435096535)

In [115]:
from chow_test import chowtest

In [116]:
chowtest(y=df[['y']].iloc[:20, :], X=df[['x1', 'x2', 'x3', 'x4']].iloc[:20, :],
         last_index_in_model_1=9,
         first_index_in_model_2=10,
         significance_level=.05)

****************************************************************************************************
Fail to reject the null hypothesis of equality of regression coefficients in the 2 periods.
****************************************************************************************************
Chow Statistic: 0.7694032344155936 p value: 0.59262
****************************************************************************************************


(0.7694032344155936, 0.5926219756968434)

# R

In [40]:
%Rpush df

In [41]:
%%R

df_train <- df[1:20, ]
df_test <- df[21:22, ]

In [42]:
%%R

m <- lm(y
~ x1 + x2 + x3 + x4, data = df_train)
print(summary(m))


Call:
lm(formula = y ~ x1 + x2 + x3 + x4, data = df_train)

Residuals:
    Min      1Q  Median      3Q     Max 
-669.00 -246.01   19.58  194.06  644.98 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) 14981.0221  4173.9545   3.589  0.00268 ** 
x1              0.4473     0.1304   3.430  0.00372 ** 
x2             36.6988    89.3775   0.411  0.68717    
x3            -19.1364    17.1235  -1.118  0.28134    
x4           -324.5281    52.8129  -6.145 1.88e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 385.3 on 15 degrees of freedom
Multiple R-squared:  0.906,	Adjusted R-squared:  0.881 
F-statistic: 36.15 on 4 and 15 DF,  p-value: 1.548e-07



In [43]:
%%R

# модель неадекватна
print(df_test['y'])
print(predict(m, newdata=df_test, interval="confidence"))
print(predict(m, newdata=df_test, interval="prediction"))

         y
20 15644.4
21 15717.8
        fit      lwr      upr
20 14250.57 13980.62 14520.53
21 14155.26 13865.55 14444.98
        fit      lwr      upr
20 14250.57 13386.00 15115.15
21 14155.26 13284.31 15026.21


In [44]:
%%R

m14 <- lm(y
~ x1 + x4, data = df_train)
print(summary(m14))


Call:
lm(formula = y ~ x1 + x4, data = df_train)

Residuals:
    Min      1Q  Median      3Q     Max 
-689.53 -217.26  -44.64  223.21  704.01 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) 14092.3334   714.1126  19.734 3.72e-13 ***
x1              0.4336     0.1128   3.844   0.0013 ** 
x4           -332.7165    44.8377  -7.420 1.00e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 411.2 on 17 degrees of freedom
Multiple R-squared:  0.8787,	Adjusted R-squared:  0.8645 
F-statistic: 61.59 on 2 and 17 DF,  p-value: 1.629e-08



In [45]:
%%R

# модель неадекватна
print(df_test['y'])
print(predict(m14, newdata=df_test, interval="confidence"))
print(predict(m14, newdata=df_test, interval="prediction"))

         y
20 15644.4
21 15717.8
        fit      lwr      upr
20 14242.08 13964.78 14519.37
21 14107.55 13887.34 14327.76
        fit      lwr      upr
20 14242.08 13331.32 15152.83
21 14107.55 13212.53 15002.58


In [46]:
%%R

df_train_1 <- df_train[1:10, ]
df_train_2 <- df_train[11:20, ]

m_1 <- lm(y
~ x1 + x2 + x3 + x4, data = df_train_1)
m_2 <- lm(y
~ x1 + x2 + x3 + x4, data = df_train_2)

rss <- deviance(m)
rss_1 <- deviance(m_1)
rss_2 <- deviance(m_2)

n_1 <- length(df_train_1[, 'x1'])
n_2 <- length(df_train_2[, 'x1'])
k <- 4


In [47]:
%%R

# H0 принимается (chow < f_critical) - выборки однородны
chow <- (rss - rss_1 - rss_2) * (n_1 + n_2 - 2 * (k + 1)) / (k + 1) / (rss_1 + rss_2)
print(chow)
f_critical <- qf(1 - 0.05, k + 1, n_1 + n_2 - 2 * (k + 1))
print(f_critical)


[1] 0.7694032
[1] 3.325835


In [49]:
%%R
library('lmtest')

resettest(m)


	RESET test

data:  m
RESET = 1.0389, df1 = 2, df2 = 13, p-value = 0.3814



# Часть 2

## Тест на длинную и короткую

In [50]:
y = df['y']
X = df[['x1', 'x2', 'x3', 'x4']]
X['const'] = 1

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.91, shuffle=False)

In [52]:
multiple_model = sm.OLS(y_train, X_train)
multiple_results = multiple_model.fit()

In [53]:
print(multiple_results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.906
Model:                            OLS   Adj. R-squared:                  0.881
Method:                 Least Squares   F-statistic:                     36.15
Date:                Sat, 11 Dec 2021   Prob (F-statistic):           1.55e-07
Time:                        02:35:14   Log-Likelihood:                -144.58
No. Observations:                  20   AIC:                             299.2
Df Residuals:                      15   BIC:                             304.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.4473      0.130      3.430      0.0

In [98]:
y = df['y']
X = df[['x4']]
X['const'] = 1

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.91, shuffle=False)

In [56]:
paired_model = sm.OLS(y_train, X_train)
paired_results = paired_model.fit()

In [57]:
print(paired_results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     61.40
Date:                Sat, 11 Dec 2021   Prob (F-statistic):           3.29e-07
Time:                        02:35:17   Log-Likelihood:                -153.39
No. Observations:                  20   AIC:                             310.8
Df Residuals:                      18   BIC:                             312.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x4          -413.0446     52.712     -7.836      0.0

### Тест на длинную и короткую регрессию

In [58]:
Markdown(fr"""
$
H_0: b_{{k-q+1}} = ... = b_k = 0, //
H_1: b_{{k-q+1}}^2 = ... = b_k^2 > 0.
$
<br><br>
$ F_{{набл}} = \frac{{R_{{UR}}^2 - R_R^2}}{{1 - R_{{UR}}^2}} \cdot \frac{{n - m}}{{q}} $<br>
$ R_{{UR}}^2 =  $ - коэффициент детерминации для длинной модели регрессии;<br>
$ R_R^2 =  $ - коэффициент детерминации для короткой модели регрессии;<br>
$ n $ - кол-во наблюдений;<br>
$ m $ - кол-во параметров длинной модели;<br>
$ q $ - кол-во удаляемых факторов.
""")


$
H_0: b_{k-q+1} = ... = b_k = 0, //
H_1: b_{k-q+1}^2 = ... = b_k^2 > 0.
$
<br><br>
$ F_{набл} = \frac{R_{UR}^2 - R_R^2}{1 - R_{UR}^2} \cdot \frac{n - m}{q} $<br>
$ R_{UR}^2 =  $ - коэффициент детерминации для длинной модели регрессии;<br>
$ R_R^2 =  $ - коэффициент детерминации для короткой модели регрессии;<br>
$ n $ - кол-во наблюдений;<br>
$ m $ - кол-во параметров длинной модели;<br>
$ q $ - кол-во удаляемых факторов.


In [59]:
rsquared_gt = multiple_results.rsquared
rsquared_lt = paired_results.rsquared
n = multiple_results.nobs
m = multiple_results.df_model + 1
q = m - (paired_results.df_model + 1)

f_value = ((rsquared_gt - rsquared_lt) / (1 - rsquared_gt)) * ((n - m) / q)
f_value

7.060842160611126

In [60]:
f_critical = stats.f.ppf(1 - ALPHA, q, n - m)

In [61]:
Markdown(fr"""
$ {f_value:.3f} > {f_critical:.3f} $<br>
$ F_{{набл}} > F_{{табл}} \rightarrow $ гипотеза $ H_1 $ принимается -
длинная модель предпочтительнее.
""")


$ 7.061 > 3.287 $<br>
$ F_{набл} > F_{табл} \rightarrow $ гипотеза $ H_1 $ принимается -
длинная модель предпочтительнее.


### Тест Рамсея

In [82]:
rs_pvalue = reset_ramsey(multiple_results, degree=3).pvalue.item()
rs_pvalue

0.3814312511749586

In [97]:
if rs_pvalue < ALPHA:
    concl = {'sign': '<', 'accept': 'отвергается', 'exist': 'есть'}
else:
    concl = {'sign': '>', 'accept': 'принимается', 'exist': 'отсутствуют'}
f'{rs_pvalue:.3f} {{sign}} {ALPHA} - H0 {{accept}} - пропущенные факторы {{exist}}'.format(**concl)

'0.381 > 0.05 - H0 принимается - пропущенные факторы отсутствуют'

# Задание 2

 1. Сравниваются две модели линейной регрессии при помощи F-теста:

In [117]:
rss_gt = 10
rss_lt = 60
n = 55
m = 4
q = 2

In [118]:
((rss_lt - rss_gt) / rss_gt) * ((n - m) / q)

127.5

 2. На основе квартальных данных с 1971 по 1976 г. с помощью МНК
получено следующее уравнение:

 3. Построить доверительный интервал для эндогенной переменной YY6 по
результатам оценивания выборки из 5 наблюдений, проверить
адекватность модели: